In [7]:
import xarray as xr
from pathlib import Path

from netCDF4 import Dataset  
import os
import time

import sys
sys.path.append('/home/links/ct715/Isca/postprocessing/plevel_interpolation/scripts')
from plevel_fn import plevel_call, daily_average, join_files, two_daily_average, monthly_average

import logging
# Set up the logging configuration
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [8]:
path = '/home/links/ct715/data_storage/isca'
exp_name = 'HS_T42_100y_60delh_t42-land'

In [6]:
# start_time=time.time()
# base_dir=os.environ['GFDL_DATA']
# exp_name_list = ['exp_name_here']
# avg_or_daily_list=['daily']
# start_file=1
# end_file=12
# nfiles=(end_file-start_file)+1

# level_set='standard'            #Default is the standard levels used previously. ssw_diagnostics are the ones blanca requested for MiMa validation

# mask_below_surface_set= ' '     # Default is to mask values that lie below the surface pressure when interpolated. 
#                                 # For some applications, you want to have values interpolated below ground, 
#                                 # i.e. as if the ground wasn't there. To use this option, this value should be set to '-x '. 


# try:
#     out_dir
# except:
#     out_dir = base_dir

# plevs={}
# var_names={}

# if level_set=='standard':

#     plevs['monthly']=' -p "1000 10000 25000 50000 85000 92500"'
#     plevs['6hourly']=' -p "1000 10000 25000 50000 85000 92500"'
#     plevs['daily']  =' -p "1000 10000 25000 50000 85000 92500"'
    
#     var_names['monthly']='-a slp height'
#     var_names['6hourly']='-a slp height'
#     var_names['daily']='-a slp height'
#     file_suffix='_interp'

# for exp_name in exp_name_list:
#     for n in range(nfiles):
#         for avg_or_daily in avg_or_daily_list:
#             print(n+start_file)

#             nc_file_in = base_dir+'/'+exp_name+'/run%04d'%(n+start_file)+'/atmos_'+avg_or_daily+'.nc'
#             nc_file_out = out_dir+'/'+exp_name+'/run%04d'%(n+start_file)+'/atmos_'+avg_or_daily+file_suffix+'.nc'

#             if not os.path.isfile(nc_file_out):
#                 plevel_call(nc_file_in,nc_file_out, var_names = var_names[avg_or_daily], p_levels = plevs[avg_or_daily], mask_below_surface_option=mask_below_surface_set)

# print('execution time', time.time()-start_time)

In [ ]:
def interp_isca_plevels(exp_name, base_dir=os.environ['GFDL_DATA'], avg_or_daily_list=['daily'], 
                        level_set='standard', mask_below_surface_set=' ', out_dir=None):
    """
    Processes netCDF files by interpolating variables to specified pressure levels.

    Parameters:
        exp_name (str): The experiment name.
        base_dir (Path): Base directory for input files.
        avg_or_daily_list (list): List of time averaging options (e.g., ['daily', 'monthly']).
        level_set (str): Level configuration ('standard' or other custom settings).
        mask_below_surface_set (str): Option for masking below surface values.
        out_dir (Path): Output directory. Defaults to `base_dir` if not provided.

    Returns:
        None
    """
    
    # Set start time
    start_time = time.time()
    
    # Get the path for the experiment directory and calculate the number of files
    exp_dir = Path(base_dir) / exp_name
    start_file = 1
    # Count number of files in the directory
    nfiles = len(list(exp_dir.glob('run*')))  # Assumes file pattern "run*"
    
    logging.info(f"Starting the file processing for {exp_name}...")

    if out_dir is None:
        out_dir = base_dir
        logging.info(f"No output directory specified, using base directory: {out_dir}")

    logging.info(f"Total files to process: {nfiles}")

    plevs = {}
    var_names = {}

    if level_set == 'standard':
        logging.info("Using standard level set.")
        plevs = {
            'monthly': ' -p "1000 10000 25000 50000 85000 92500"',
            '6hourly': ' -p "1000 10000 25000 50000 85000 92500"',
            'daily': ' -p "1000 10000 25000 50000 85000 92500"'
        }
        var_names = {
            'monthly': '-a slp height',
            '6hourly': '-a slp height',
            'daily': '-a slp height'
        }
        file_suffix = '_interp'
    else:
        raise ValueError("Unsupported level_set: only 'standard' is implemented.")

    # Processing the files
    for n in range(nfiles):
        for avg_or_daily in avg_or_daily_list:
            nc_file_in = exp_dir / f"run{n + start_file:04d}" / f"atmos_{avg_or_daily}.nc"
            nc_file_out = Path(out_dir) / exp_name / f"run{n + start_file:04d}" / f"atmos_{avg_or_daily}{file_suffix}.nc"

            if not nc_file_out.is_file():
                logging.info(f"Interpolating file: {nc_file_in} -> {nc_file_out}")
                try:
                    plevel_call(
                        nc_file_in,
                        nc_file_out,
                        var_names=var_names[avg_or_daily],
                        p_levels=plevs[avg_or_daily],
                        mask_below_surface_option=mask_below_surface_set
                    )
                    logging.info(f"Successfully interpolated and saved to: {nc_file_out}")
                except Exception as e:
                    logging.error(f"Error during interpolation of {nc_file_in}: {e}")
            else:
                logging.info(f"File already exists, skipping: {nc_file_out}")

    logging.info('File processing completed.')
    logging.info(f'Execution time: {time.time() - start_time} seconds')

In [11]:
time.time()

1733931046.306213